In [ ]:
pip install gluonts mxnet matplotlib pandas


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gluonts.dataset.common import ListDataset
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions

In [ ]:
# Simulate 1 year (weekly data)
date_range = pd.date_range("2020-01-01", periods=52, freq="W")

In [ ]:
# Create synthetic target data
data = {
    "start": str(date_range[0]),
    "target": (10 + np.sin(np.arange(52) / 3.0) * 2 + np.random.normal(0, 0.5, 52)).tolist()
}

train_data = ListDataset([data], freq="W", one_dim_target=True)

In [ ]:

estimator = DeepAREstimator(
    freq="W",
    prediction_length=4,  # Forecast 4 weeks ahead
    trainer=Trainer(epochs=30)
)

predictor = estimator.train(train_data)

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_data,
    predictor=predictor,
    num_samples=100
)

In [ ]:
for forecast, ts in zip(forecast_it, ts_it):
    ts[-20:].plot(label="actual")
    forecast.plot(prediction_intervals=[50, 90])
    plt.legend()
    plt.grid()
    plt.show()

Transform Data to GluonTS Format
Here’s how to prepare ListDataset for GluonTS with covariates:

In [ ]:
import pandas as pd
from gluonts.dataset.common import ListDataset


In [ ]:
# Load your dataframe
df = pd.read_csv("your_data.csv", parse_dates=["date"])

# Ensure data is sorted
df = df.sort_values(["farm_id", "date"])

In [ ]:
data_list = []

for farm_id, group in df.groupby("farm_id"):
    group = group.sort_values("date")
    
    start = group["date"].iloc[0]
    target = group["yield"].tolist()
    
    dynamic_feat = [
        group["rainfall"].tolist(),      # Feature 1
        group["temperature"].tolist()    # Feature 2
    ]

    data_list.append({
        "start": str(start),
        "target": target,
        "dynamic_feat": dynamic_feat,
        "item_id": farm_id
    })

train_ds = ListDataset(data_list, freq="W")

In [ ]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer

estimator = DeepAREstimator(
    freq="W",
    prediction_length=4,
    trainer=Trainer(epochs=30),
    use_feat_dynamic_real=True
)

predictor = estimator.train(train_ds)

In [ ]:
from gluonts.evaluation.backtest import make_evaluation_predictions

forecast_it, ts_it = make_evaluation_predictions(
    dataset=train_ds,
    predictor=predictor,
    num_samples=100
)

In [ ]:
import matplotlib.pyplot as plt
for forecast, ts in zip(forecast_it, ts_it):
    ts[-20:].plot(label="actual")
    forecast.plot(prediction_intervals=[50, 90])
    plt.legend()
    plt.grid()
    plt.show()